In [1]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import datetime, timedelta
import datetime as dt
from pymongo import MongoClient
import pprint
import csv
host_ip = "192.168.1.105"

In [2]:
client = MongoClient(host_ip, 27017)  # connect to mongo client
db = client.fit3182
climate_db= db["climate_db"]


In [3]:


def publish_message(producer_instance, topic_name, key, data): # publish message
    try:
        key_bytes = bytes(key, encoding='utf-8') # encode as utf-8
        producer_instance.send(topic_name, key=key_bytes, value=data)  # encode as utf-8 and send the message 
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],# send to a specific server
                                  value_serializer=lambda x:dumps(x).encode('ascii'), # for encoding as  ascii
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    


In [4]:
def ForSeenClimateData(filename): # function to send hotspot data
     with open(filename, 'r') as csvfile:
        data = csv.DictReader(csvfile)
        prep_data = [] #convert csv to dict 
        for row in data:
           
            row['publisher']=1   #only this publisher will send climate report

            row['station']="98740%d" % random.randrange(0,2) # from a random station    
            prep_data.append(row)

        return prep_data

In [5]:
def currentDate():
    latestDateR = None  # Initialize the variable outside the loop
    # to get the lastest date
    lastestDate = climate_db.aggregate([
        {"$project": {"date": 1}},
        {"$sort": {"date": 1}},
        {"$limit": 1}
    ])

    for document in lastestDate:
        latestDateR = document["date"]
        
        
        return latestDateR  # return latest_dateR to get the lastest date


In [6]:
def incrementDate(dateSt):
    
    date = datetime.strptime(dateSt, "%d/%m/%Y")

    # increment the date by one day
    next_day = date + timedelta(days=1)

    
    print("Original Date:", date.strftime("%d/%m/%Y"))
    print("Original Date:", next_day.strftime("%d/%m/%Y"))
    return next_day.strftime("%d/%m/%Y")

In [7]:




time=10 # to send a climate report in 10 sec(A day)
if __name__ == '__main__':
    topic = 'Lastest_Climate'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    newClimateSet = ForSeenClimateData('climate_streaming.csv')
    curDate=currentDate()
    random.seed(32939375)
    while True:
        data = newClimateSet[(random.randrange(0,len(newClimateSet)))]
        data['date']=curDate
      
        publish_message(producer, topic, 'jsondata', data)
        curDate=incrementDate(curDate)
        
        sleep(time)

Publishing records..
Message published successfully. Data: {'latitude': '-36.2693', 'longitude': '143.1768', 'air_temperature_celcius': '12', 'relative_humidity': '46.6', 'windspeed_knots': '6.7', 'max_wind_speed': '14', 'precipitation ': ' 0.00G', 'GHI_w/m2': '106', 'publisher': 1, 'station': '987400', 'date': '01/02/2024'}
Original Date: 01/02/2024
Original Date: 02/02/2024
Message published successfully. Data: {'latitude': '-37.469', 'longitude': '148.125', 'air_temperature_celcius': '14', 'relative_humidity': '48.4', 'windspeed_knots': '7.5', 'max_wind_speed': '13', 'precipitation ': ' 0.00G', 'GHI_w/m2': '122', 'publisher': 1, 'station': '987400', 'date': '02/02/2024'}
Original Date: 02/02/2024
Original Date: 03/02/2024
Message published successfully. Data: {'latitude': '-36.943', 'longitude': '143.286', 'air_temperature_celcius': '24', 'relative_humidity': '55.2', 'windspeed_knots': '5.8', 'max_wind_speed': '11.1', 'precipitation ': ' 0.00I', 'GHI_w/m2': '197', 'publisher': 1, 's

KeyboardInterrupt: 